# import

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 51.2 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 10.3 MB/s eta 0:00:00
  Obtaining dependency information for dask from https://files.pythonhosted.org/packages/91/03/d3202129672be2931b3dad7407b24e98c0b847f7b158ffdf88d34cdcbbca/dask-2023.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for cloudpickle>=1.5.0 from https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec>=2021.09.0 from https://files.pythonhosted.org/packages/96/0e/2be9b5a2e3f736577e749bbdf27a1e7e965041e1c908d49dedf56eeb2b8a/fsspec-2023.12.1-py3-none-any.whl.metadata
  Obtain

In [2]:
import numpy as np
import pandas as pd
import arrow
import joblib
import openpyxl
import pickle
import os
from datetime import datetime, date, time, timedelta
import random
import copy
import time 
import csv
import json

# main pre-processing 

looping through and shrinking pkl files and storing in another directory

In [9]:
# Directory where the data is located
data_directory = './Data/'

# Iterate through subfolders D02 to D25
for subfolder in range(25, 26):
    subfolder_name = f'D{subfolder:02d}'
    print(subfolder_name)
    subfolder_path = os.path.join(data_directory, subfolder_name)
    
    # Check if the subfolder exists
    if os.path.exists(subfolder_path):
        # Iterate through files in the subfolder

        dfs = []

        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            print(file_path)
            
            # Check if the file is a pickle file
            if file_name.endswith('.pkl'):
                print(file_name)
                # Load the pickle file and append it to the list of dataframes
                try:
                    with open(file_path, 'rb') as file:
                        data = pickle.load(file)
                        dfs.append(data)
                except EOFError:
                    print("EOFError: Failed to load data from the file.")
                except Exception as e:
                    print("An error occurred:", e)
        
        # Concatenate all dataframes into a single dataframe
        df = pd.concat(dfs, ignore_index=True)

        df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
        df['day'] = df['date'].dt.date

        # create source of retweet action
        df['source'] = df['user'].apply(lambda x: x['screenName'] if x is not None and 'screenName' in x else None)

        # Get target
        df['target'] = df['retweet'].apply(lambda x: x['user']['screenName'] if (x and 'user' in x and 'screenName' in x['user']) else None)

        df['weight'] = 1

        # removing None - where there is just a tweet and no retweet
        df = df.dropna(subset=['target'])

        df_less_cols = df.copy()

        df_less_cols = df_less_cols[['date','source','target','weight','day']]

        output_directory = './data_pr_rt_2/'
        os.makedirs(output_directory, exist_ok=True)

        # Assuming df_less_cols is your DataFrame
        df_less_cols.to_pickle(os.path.join(output_directory, f'D{subfolder:02d}.pkl'))  


D25
./Data/D25/election_v2_2022_APR_D25_21_02d_20231021.pkl
election_v2_2022_APR_D25_21_02d_20231021.pkl
./Data/D25/election_v2_2022_APR_D25_19_02d_20231021.pkl
election_v2_2022_APR_D25_19_02d_20231021.pkl
./Data/D25/election_v2_2022_APR_D25_22_02d_20231021.pkl
election_v2_2022_APR_D25_22_02d_20231021.pkl
./Data/D25/election_v2_2022_APR_D25_01_01_20231020.pkl
election_v2_2022_APR_D25_01_01_20231020.pkl
./Data/D25/election_v2_2022_APR_D25_08_08_20231020.pkl
election_v2_2022_APR_D25_08_08_20231020.pkl
./Data/D25/election_v2_2022_APR_D25_07_07_20231020.pkl
election_v2_2022_APR_D25_07_07_20231020.pkl
./Data/D25/election_v2_2022_APR_D25_09_09_20231020.pkl
election_v2_2022_APR_D25_09_09_20231020.pkl
./Data/D25/election_v2_2022_APR_D25_12_02d_20231020.pkl
election_v2_2022_APR_D25_12_02d_20231020.pkl
./Data/D25/election_v2_2022_APR_D25_13_02d_20231020.pkl
election_v2_2022_APR_D25_13_02d_20231020.pkl
./Data/D25/election_v2_2022_APR_D25_05_05_20231020.pkl
election_v2_2022_APR_D25_05_05_20231020.

#  preprocessing (repaired)

In [4]:
# Directory where the data is located
data_directory = './data_pr_rt_recollected/Recollected/'


# Iterate through subfolders D02 to D25
for subfolder in range(19, 21):
    subfolder_name = f'D{subfolder:02d}'
    print(subfolder_name)
    subfolder_path = os.path.join(data_directory, subfolder_name)
    
    # Check if the subfolder exists
    if os.path.exists(subfolder_path):
        # Iterate through files in the subfolder

        dfs = []

        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            print(file_path)
            
            # Check if the file is a pickle file
            if file_name.endswith('.pkl'):
                print(file_name)
                # Load the pickle file and append it to the list of dataframes
                try:
                    with open(file_path, 'rb') as file:
                        data = pickle.load(file)
                        dfs.append(data)
                except EOFError:
                    print("EOFError: Failed to load data from the file.")
                except Exception as e:
                    print("An error occurred:", e)
        
        # Concatenate all dataframes into a single dataframe
        df = pd.concat(dfs, ignore_index=True)

        df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
        df['day'] = df['date'].dt.date

        # create source of retweet action
        df['source'] = df['user'].apply(lambda x: x['screenName'] if x is not None and 'screenName' in x else None)

        # Get target
        df['target'] = df['retweet'].apply(lambda x: x['user']['screenName'] if (x and 'user' in x and 'screenName' in x['user']) else None)

        df['weight'] = 1

        df = df.copy()

        # df = df.drop_duplicates()

        # removing None - where there is just a tweet and no retweet
        df = df.dropna(subset=['target'])

        df_less_cols = df.copy()

        df_less_cols = df_less_cols[['date','source','target','weight','day']]

        output_directory = './data_pr_rt/'
        os.makedirs(output_directory, exist_ok=True)

        # Assuming df_less_cols is your DataFrame
        df_less_cols.to_pickle(os.path.join(output_directory, f'D{subfolder:02d}.pkl'))


        


D19
D20
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_02_02_20231020.pkl
election_v2_2022_APR_D20_02_02_20231020.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_19_02d_20231025.pkl
election_v2_2022_APR_D20_19_02d_20231025.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_08_08_20231020.pkl
election_v2_2022_APR_D20_08_08_20231020.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_11_02d_20231020.pkl
election_v2_2022_APR_D20_11_02d_20231020.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_21_02d_20231025.pkl
election_v2_2022_APR_D20_21_02d_20231025.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_18_02d_20231020.pkl
election_v2_2022_APR_D20_18_02d_20231020.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_06_06_20231020.pkl
election_v2_2022_APR_D20_06_06_20231020.pkl
./data_pr_rt_recollected/Recollected/D20/election_v2_2022_APR_D20_03_03_20231020.

In [ ]:
# Index(['affects', 'category', 'contentClassification', 'date', 'emotion',
#        'engagementScore', 'externalUrl', 'focuses', 'geo', 'hashtag',
#        'highlights', 'id', 'idStr', 'impression', 'inReplyToStatusId',
#        'inReplyToStatusIdStr', 'isQuote', 'isRT', 'keyphrase', 'keyword',
#        'lang', 'media', 'mentions', 'origin', 'permaSlink', 'quote', 'radar',
#        'reach', 'retweet', 'score', 'story', 'text', 'tone', 'uid', 'user',
#        'month', 'year'],
#       dtype='object')

# Importing processed tweets 

concats all the pkl files by day into large file

In [3]:
folder_path = './data_pr_rt_2/'
file_list = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]
dfs = []

for file in file_list:
    file_path = os.path.join(folder_path, file)
    # Assuming the data is stored as a DataFrame in each .pkl file
    df = pd.read_pickle(file_path)
    dfs.append(df)
    print(file)

df = pd.concat(dfs, ignore_index=True)

# Now, concatenated_df contains the data from all .pkl files in the folder
print(df.head())  # Print the first few rows of the concatenated DataFrame


D14.pkl
D06.pkl
D09.pkl
D07.pkl
D13.pkl
D02.pkl
D17.pkl
D11.pkl
D20.pkl
D05.pkl
D23.pkl
D25.pkl
D03.pkl
D08.pkl
D19.pkl
D18.pkl
D24.pkl
D22.pkl
D21.pkl
D15.pkl
D04.pkl
D16.pkl
D10.pkl
D01.pkl
                       date       source          target  weight         day
0 2022-04-14 05:00:00+00:00     ARNHED71        CBarrely       1  2022-04-14
1 2022-04-14 05:00:00+00:00  DoubleVPenn   2022Elections       1  2022-04-14
2 2022-04-14 05:00:00+00:00    Titians20    BOROWSKIMIKE       1  2022-04-14
3 2022-04-14 05:00:00+00:00   jcmartin87  HamsterFreedom       1  2022-04-14
4 2022-04-14 05:00:00+00:00     bert6691  LaurianeTroise       1  2022-04-14


In [4]:
df.head()

,date,source,target,weight,day
0,2022-04-14 05:00:00+00:00,ARNHED71,CBarrely,1,2022-04-14
1,2022-04-14 05:00:00+00:00,DoubleVPenn,2022Elections,1,2022-04-14
2,2022-04-14 05:00:00+00:00,Titians20,BOROWSKIMIKE,1,2022-04-14
3,2022-04-14 05:00:00+00:00,jcmartin87,HamsterFreedom,1,2022-04-14
4,2022-04-14 05:00:00+00:00,bert6691,LaurianeTroise,1,2022-04-14


In [5]:
len(df.target.unique())

160966

In [16]:
len(df)

16089093

In [14]:
# Print the number of rows before removing duplicates
print("Number of rows before removing duplicates:", len(df))

# Remove duplicates
df_no_dup = df.drop_duplicates()

# Print the number of rows after removing duplicates
print("Number of rows after removing duplicates:", len(df_no_dup))

# Print the number of duplicates removed
duplicates_removed = len(df) - len(df_no_dup)
print("Number of duplicates removed:", duplicates_removed)

Number of rows before removing duplicates: 16089093
Number of rows after removing duplicates: 16073433
Number of duplicates removed: 15660


In [7]:
# save to rt_net to csv  
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# unfiltered dataset (date,source,target,weight,day)
excel_file_path = os.path.join(parent_directory, 'data', 'load', 'rt_net_unfiltered_2023-12-11_dup.csv')
df.to_csv(excel_file_path, index=False,header=False)

# filtered (source,target,weight)
df_filtered_2 = df[['source','target','weight']]
excel_file_path = os.path.join(parent_directory, 'data', 'load', 'rt_net_filtered_2023-12-11_dup.csv')
df_filtered_2.to_csv(excel_file_path, index=False,header=False)

# filtered and summed 
df_filtered_3 = df.groupby(['source', 'target'])['weight'].sum().reset_index()
excel_file_path = os.path.join(parent_directory, 'data', 'load', 'rt_net_filtered_weighted_2023-12-11_dup.csv')
df_filtered_3.to_csv(excel_file_path, index=False,header=False)


### Load opinion leaders list (gross)

In [5]:
# Read the 'all_mps' sheet from the Excel file
df_elites = pd.read_excel('./input/Elite actors.xlsx', sheet_name='all_mps')
df_elites.head()

,Duplicate?,Handle,Twitter URL,Position,Affilation,Comment
0,False,PapNdiaye,https://twitter.com/PapNdiaye,Government,LREM,NaN
1,False,sretailleau,https://twitter.com/sretailleau,Government,LREM,NaN
2,False,RimaAbdulMalak,https://twitter.com/RimaAbdulMalak,Government,LREM,NaN
3,False,E_DupondM,https://twitter.com/E_DupondM,Government,LREM,NaN
4,False,MinColonna,https://twitter.com/MinColonna,Government,LREM,NaN


In [6]:
df_elites.Affilation.value_counts()

LREM    294
RN      105
LR       98
LFI      73
EELV     17
RQ       11
Name: Affilation, dtype: int64

In [7]:
df_elites.Affilation.value_counts().sum()

598

In [ ]:
# Read the 'all_mps' sheet from the Excel file
df_constituencies = pd.read_excel('./input/Elite actors.xlsx', sheet_name='constituency_results')
df_constituencies.head()

### How many elites are present in our initial dataset?

In [8]:
# Extract the 'twitter_handle' values from df_elites and filtered_targets
twitter_handles_in_df = df_elites['Handle']
twitter_handles_in_target = df['target']

# Find twitter handles that are in the target list
handles_found_in_target = twitter_handles_in_df[twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

# Find twitter handles that are not in the target list
handles_not_found_in_target = twitter_handles_in_df[~twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

print("Handles found in the target list:", len(handles_found_in_target))
print("Handles not found in the target list:", len(handles_not_found_in_target))
# print(*handles_found_in_target, sep=' ')


Handles found in the target list: 408
Handles not found in the target list: 190


# filter by influence 

min 1000 retweets - this is used to find out how many opinion leaders are left in there, but the final dataset won't remove these

In [58]:
####### filter by influence (1000 rt)
df_influential_users = df.copy()

df_influential_users = df_influential_users[df_influential_users.groupby('target')['weight'].transform('sum') > 400]
df_influential_users = df_influential_users.reset_index(drop=True)


In [59]:
print('there are ' + str(len(df) - len(df_influential_users)) + ' less tweets after filtering out non-influential users <500 retweets')

there are 2956610 less tweets after filtering out non-influential users <500 retweets


In [60]:
print('there are ' + str(len(df.target.unique()) - len(df_influential_users.target.unique())) + ' less elites after filtering out non-influential users <1000 retweets, and now a total of ' + str(len(df_influential_users.target.unique())) + ' unique users left, from ' + str(len(df.target.unique()))) 

there are 166961 less elites after filtering out non-influential users <1000 retweets, and now a total of 4696 unique users left, from 171657


In [61]:
# find out how many MPs are influential 

# Extract the 'twitter_handle' values from df_elites and filtered_targets
twitter_handles_in_df = df_elites['Handle']
twitter_handles_in_target = df_influential_users['target']

# Find twitter handles that are in the target list
handles_found_in_target = twitter_handles_in_df[twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

# Find twitter handles that are not in the target list
handles_not_found_in_target = twitter_handles_in_df[~twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

print("Politicians who are influential:", len(handles_found_in_target))
print("Politicians who are not influential:", len(handles_not_found_in_target))

Politicians who are influential: 128
Politicians who are not influential: 470


In [62]:
df_elites[df_elites['Handle'].isin(handles_found_in_target)]['Affilation'].value_counts()

LREM    54
LFI     27
RN      25
LR      11
RQ       7
EELV     4
Name: Affilation, dtype: int64

# filter by participation rate

still looking for opinion leaders who are left in after participation rate reduction

In [63]:
####### filter by participation rate
# Group by 'day' and 'target' and count the occurrences
target_counts_per_day = df_influential_users.groupby(['day', 'target']).size().reset_index(name='count')
# Get the total number of unique days in the dataset
# total_days = df_influential_users['day'].nunique()

total_days = 17 # 70% participation rate 
# Filter targets that appear at least once every day
filtered_targets = target_counts_per_day.groupby('target').filter(lambda x: len(x) >= total_days)

In [64]:
filtered_targets.head()

,day,target,count
0,2022-04-01,100drine_Juste,3
1,2022-04-01,13WolfLee,2
2,2022-04-01,17_so2020,5
3,2022-04-01,1968Framboisine,2
4,2022-04-01,1ElisaMoreno,24


In [65]:
len(filtered_targets)

61914

### find opinion leaders from gross list

In [66]:
# Extract the 'twitter_handle' values from df_elites and filtered_targets
twitter_handles_in_df = df_elites['Handle']
twitter_handles_in_target = filtered_targets['target']

# Find twitter handles that are in the target list
handles_found_in_target = twitter_handles_in_df[twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

# Find twitter handles that are not in the target list
handles_not_found_in_target = twitter_handles_in_df[~twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

print("Handles found in the target list:", len(handles_found_in_target))
print("Handles not found in the target list:", len(handles_not_found_in_target))
print(*handles_found_in_target, sep=' ')


Handles found in the target list: 119
Handles not found in the target list: 479
CZacharopoulou AgnesRunacher ChristopheBechu al_petel Elisabeth_Borne franckriester CBeaune GabrielAttal alexiscorbiere SebLecornu olivierdussopt GDarmanin alma_dufour barbarapompili BrunoLeMaire FJolivet36 RolandLescure AdeMontchalin CoDubost OlgaGivernet AVFreschi LaetitiaAvia MarieLebec78 Sebastien_Nadot EmilieCChalas AChristine_Lang GillesLeGendre moreaujb23 BrigBourguignon Aurelientache ebothorel RichardFerrand trudigoz BQuestel SachaHoulie SylvainMaillard mguevenoux EliseFajgeles CCastaner jmzulesi saidahamada sarahelhairy auroreberge EmmanuelMacron avecvous MLP_officiel BlairyEmmanuel PhOlivierRN HeleneLaporteRN diaz_edwige M_Dauchy A_Masson06 DominiqueBilde v_joron JLechanteux ljacobelli BrunoBilde J_Bardella LaureLavalette BallardPhilippe NMeizonnet sebchenu JulienOdoul JphTanguy franckallisio YoannGillet30 jllacapelle JPGarraud ThierryMARIANI ZemmourEric MarionMarechal NicolasBay_ stanislasrig Dam

In [67]:
df_elites_opinion_leaders_present = df_elites[df_elites['Handle'].isin(handles_found_in_target)]

In [68]:
df_elites_opinion_leaders_present.Affilation.value_counts()

LREM    50
LFI     25
RN      24
LR      10
RQ       6
EELV     4
Name: Affilation, dtype: int64

In [74]:
df_elites_opinion_leaders_present.rename(columns={'Affilation': 'Affiliation'}, inplace=True)

/tmp/ipykernel_198/2480142810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elites_opinion_leaders_present.rename(columns={'Affilation': 'Affiliation'}, inplace=True)


In [126]:
selected_columns = ['Affiliation', 'Handle','Position']
df_selected = df_elites_opinion_leaders_present[selected_columns].copy()

# Rename affiliations as specified
affiliation_mapping = {'LREM': 'Ens', 'LFI': 'NUPES', 'RN': 'RN', 'LR': 'LR', 'EELV': 'NUPES'}
df_selected['Affiliation'] = df_selected['Affiliation'].map(affiliation_mapping)
# Remove rows where affiliation is 'RQ'
df_filtered = df_filtered[(df_filtered['Affiliation'] == 'Ens') | (df_filtered['Affiliation'] == 'RN') | (df_filtered['Affiliation'] == 'LR') | (df_filtered['Affiliation'] == 'NUPES')]

df_filtered = df_filtered.reset_index(drop=True)

In [127]:
df_filtered

,Affiliation,Handle,Position
0,Ens,CZacharopoulou,Government
1,Ens,AgnesRunacher,Government
2,Ens,ChristopheBechu,Government
3,Ens,al_petel,MP
4,Ens,Elisabeth_Borne,Government
...,...,...,...
108,LR,oliviermarleix,MP
109,LR,brigitte_kuster,Ex-MP
110,LR,ChJacob77,Ex-MP
111,LR,JulienAubert84,Ex-MP


In [128]:
# df_elites_opinion_leaders_present[df_elites_opinion_leaders_present['Affilation']=='RN']

In [129]:
import os
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
excel_file_path = os.path.join(parent_directory, 'data', 'twitter_handles', 'opinion_leaders_113.xlsx')
df_filtered.to_excel(excel_file_path, index=False)

In [131]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
excel_file_path = os.path.join(parent_directory, 'data', 'twitter_handles', 'opinion_leaders_113.csv')
df_filtered['Handle'].to_csv(excel_file_path, index=False,header=False)

In [130]:
len(df_filtered['Handle'].unique())

113